In [1]:
# Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [3]:
# Set Parameters
IMG_SIZE = (128, 128) ## to resize all images to 128x128 pixels
BATCH_SIZE = 32 ## process 32 images at a time during training
EPOCHS = 2 ## train the model for 2 full passes over the dataset ie showing the each image twice to the model.

Here we resized the image to 128x128 pixels for consistency. The model will process the data in small groups (batches) of 32 images to efficiently use memory and improve training speed. The training will run for 2 epochs, meaning the entire dataset will be used twice during the training process.

In [5]:
# Prepare Image Data (creates an image generator that normalizes pixel values and splits data for training and validation)
datagen = ImageDataGenerator(
    rescale=1./255, ## This divides every pixel value in the image by 255
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    "boat_data/boat_data/train",       # make sure this folder contains gondola, motorboat, ferry
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    #subset='training'
) ## The above loads and preprocesses training images from the boat_data folder

val_gen = datagen.flow_from_directory(
    "boat_data/boat_data/test",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    #subset='validation'
) ## While this one loads and preprocesses validation images from the same folder


Found 4774 images belonging to 24 classes.
Found 4774 images belonging to 24 classes.


Here we normalized the image pixel values to a 0–1 range and split the dataset into 80% training and 20% validation. Images were loaded from the boat_data folder, resized to 128x128 pixels, and grouped into batches of 32. The data generator automatically assigned labels based on folder names (e.g., gondola, motorboat, ferry) using categorical encoding

In [77]:
# Build CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(2, 2), ## specifies that the activation function used within a layer is the Rectified Linear Unit (ReLU). 
                        ## This means that the neuron will output the input directly if it's positive, and zero if it's negative

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'), 
    Dropout(0.5), 
    Dense(train_gen.num_classes, activation='softmax')  
]) ## # Output layer 24 for all the 24 classes.


Here we built a Convolutional Neural Network (CNN) to classify boat images their categories. The model includes convolutional and pooling layers to extract image features, followed by flattening and dense layers to make predictions. A dropout layer was added to reduce overfitting, and the final softmax layer outputs probabilities for each of the three classes

In [80]:
# Compile Model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy']) ## Prepares the model for training using the Adam optimizer, categorical crossentropy loss 
                                    ## (for multi-class classification), and accuracy as the evaluation metric

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 24)             │         3,096 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,395,416 (28.21 MB)

 Trainable params: 7,395,416 (28.21 MB)

 Non-trainable params: 0 (0.00 B)

Here we compiled the CNN model using the Adam optimizer for adaptive learning, with categorical crossentropy as the loss function suitable for multi-class classification. Accuracy was chosen to track model performance during training. The model.summary() command gives a quick overview of the model’s structure and total trainable parameters

In [83]:
# Train Model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS
)

Epoch 1/2
150/150 ━━━━━━━━━━━━━━━━━━━━ 277s 2s/step - accuracy: 0.4071 - loss: 2.2922 - val_accuracy: 0.6267 - val_loss: 1.2881
Epoch 2/2
150/150 ━━━━━━━━━━━━━━━━━━━━ 629s 4s/step - accuracy: 0.6087 - loss: 1.4028 - val_accuracy: 0.7392 - val_loss: 0.9277


Here we trained the CNN model using the prepared training data and validated its performance with the validation set. The model learned over 2 epochs, adjusting its internal parameters to reduce classification errors. Training progress, including accuracy and loss for both training and validation, was stored in the history variable for future analysis or visualization.

In [85]:
# Path to the image you want to predict
img_path = "boat_data/image1.jpg"  # Replace with your image path

# Preprocess the image
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make prediction
pred = model.predict(img_array)

# Get class labels (based on the train_gen class indices)
class_labels = {v: k for k, v in train_gen.class_indices.items()}  # Inverse mapping

# Print predicted class
predicted_class = class_labels[np.argmax(pred)]  # Get class with the highest probability
print(f"Predicted class: {predicted_class} ")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted class: Lanciafino10mBianca 


Here we loaded and preprocessed an image for prediction. The image was resized, converted to an array, and normalized. Finally, the model then predicted the class as **Lanciafino10mBianca**

In [88]:
# Path to the image you want to predict
img_path = "boat_data/image.jpg"  # Replace with your image path

# Preprocess the image
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Rescale pixel values to [0, 1]

# Make prediction
pred = model.predict(img_array)

# Get class labels (based on the train_gen class indices)
class_labels = {v: k for k, v in train_gen.class_indices.items()}  # Inverse mapping

# Print predicted class
predicted_class = class_labels[np.argmax(pred)]  # Get class with the highest probability
print(f"Predicted class: {predicted_class} ")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 626ms/step
Predicted class: Patanella 


We tried another image and it predicted it to be **Patanella**